## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-03-40-32 +0000,nypost,Two Venezuelan military aircraft flew over US ...,https://nypost.com/2025/09/04/us-news/two-vene...
1,2025-09-05-03-32-18 +0000,nypost,Trump announces Delta Air Lines will officiall...,https://nypost.com/2025/09/04/us-news/trump-an...
2,2025-09-05-03-27-29 +0000,bbc,Gamers frustrated as Hollow Knight: Silksong c...,https://www.bbc.com/news/articles/ckgejpxzepko...
3,2025-09-05-03-25-28 +0000,nyt,Melania Trump Has a Warning for Humanity: ‘The...,https://www.nytimes.com/2025/09/04/us/politics...
4,2025-09-05-03-10-58 +0000,nyt,"Justice Amy Coney Barrett Argues Her Own Case,...",https://www.nytimes.com/2025/09/04/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,68
45,court,14
145,out,14
149,jr,14
16,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
238,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...,159
32,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,146
42,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...,137
10,2025-09-05-02-44-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,136
267,2025-09-04-15-00-00 +0000,wsj,The BLS will publish its first jobs report Fri...,https://www.wsj.com/economy/jobs/jobs-report-a...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
238,159,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
120,68,2025-09-04-21-25-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
122,64,2025-09-04-21-22-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
227,63,2025-09-04-16-36-57 +0000,nypost,CBS News staffers ‘freaking out’ as new Paramo...,https://nypost.com/2025/09/04/media/paramount-...
184,59,2025-09-04-18-50-00 +0000,wsj,The Justice Department opened a criminal inves...,https://www.wsj.com/politics/policy/lisa-cook-...
100,55,2025-09-04-21-54-34 +0000,nyt,"After Boat Strike, Rubio Says U.S. Will Help O...",https://www.nytimes.com/2025/09/04/world/ameri...
131,43,2025-09-04-21-07-20 +0000,nyt,Portugal Searches for Cause of Lisbon Funicula...,https://www.nytimes.com/2025/09/04/world/europ...
280,42,2025-09-04-14-00-00 +0000,wsj,Benjamin Franklin arrived in France in 1776 to...,https://www.wsj.com/politics/policy/ben-frankl...
352,37,2025-09-04-08-26-02 +0000,bbc,Allies ready to support Ukraine before and aft...,https://www.bbc.com/news/articles/c5y85x58nj4o...
59,36,2025-09-04-23-05-58 +0000,nypost,Giorgio Armani invented the Hollywood power pl...,https://nypost.com/2025/09/04/world-news/giorg...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
